# Обучения языковых моделей

В этом ноутбуке будет произведено обучение языковых моделей для модели итеративного исправления. Требуется обучить две модели:

1. Слева-направо
2. Справа-налево

В качестве обучающего корпуса будет взят фрагмент корпуса Тайга, а именно части из соц.сетей, новостных сайтов, субтитров, так как это должен быть достаточно близкий к изучаемому домен.

В качестве модели было решено взять KenLM.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
import os
import string
import re
from collections import Counter
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd

import nltk

from IPython.display import display
from tqdm.notebook import tqdm

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
CONFIGS_PATH = os.path.join(PROJECT_PATH, 'src', 'configs')
os.environ['DP_PROJECT_PATH'] = PROJECT_PATH

## Данные

В качестве данных для обучения решено было взять фрагмент корпуса [Тайга](https://tatianashavrina.github.io/taiga_site/). Были выбраны разделы:
1. Новости
2. Соцсети
3. Субтитры

Все файлы для скачивания доступны по [ссылке](https://tatianashavrina.github.io/taiga_site/downloads) в разделе "Our special collections for".

## Подготовка

В первую очередь требуется предобработать все тексты, что у нас имеются. Согласно задаче, нас не интересует регистр слов и пунктуация, поэтому избавимся от нее. В качестве результата должны получиться два текстовых файла (прямой и обратный), где каждое предложение расположено на отдельной строчке -- именно в таком виде следует подавать данные для обучающей программы.

In [5]:
!mkdir ../data/processed/kenlm -p

In [6]:
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
TAIGA_PATH = os.path.join(DATA_PATH, 'external', 'taiga')
RESULT_PATH = os.path.join(DATA_PATH, 'processed', 'kenlm')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

### Новости

Начнем с обработки новостей. Каждую из них требуется разбить на предложения и токенизировать, избавившись от пунктуации.

In [ ]:
NEWS_PATH = os.path.join(TAIGA_PATH, 'news')
news_texts = []

Сначала обработаем только Фонтанку, потому что там есть деление по годам.

In [ ]:
fontanka_path = os.path.join(NEWS_PATH, 'Fontanka', 'texts')
for year in tqdm(sorted(os.listdir(fontanka_path))):
    year_path = os.path.join(fontanka_path, year)
    for filename in sorted(os.listdir(year_path)):
        with open(os.path.join(year_path, filename), 'r') as inf:
            news_texts.append(inf.read())

Теперь обработаем тексты по всем остальным новостным сайтам.

In [ ]:
for source in tqdm(sorted(os.listdir(NEWS_PATH))):
    if source == 'Fontanka':
        continue
    texts_path = os.path.join(NEWS_PATH, source, 'texts')
    for filename in sorted(os.listdir(texts_path)):
        with open(os.path.join(texts_path, filename), 'r') as inf:
            news_texts.append(inf.read())

Разобьем каждый текст по предложениям.

In [ ]:
news_sentences = []
for text in tqdm(news_texts):
    news_sentences += [
        x.lower() for x in nltk.tokenize.sent_tokenize(
            text, language='russian'
        )
    ]

Теперь каждый текст разобьем на токены и избавимся от тех из них, которые отвечают за пунктуацию. Сначала надо изучить какие вообще символы встречаются в текстах, чтобы понять что из этого может быть пунктуацией (иначе мы можем не учесть какие-то специфичные символы).

In [ ]:
characters = Counter()
for sentence in tqdm(news_sentences):
    characters.update(list(sentence))

In [ ]:
characters

1. Беспокоит наличие символов `\n`, `\t`. Чтобы избавиться от них заменим `\n`, `\t` на пробел.
2. Наличие иностранных символов можно объяснить ссылкой на какой-то иностранный источник или имя на оригинальном языке.
3. Поиском предложений с соответствующими символами удавалось обнаружить очень "шумные" предложения.

Пример шумного предложения.

In [ ]:
news_sentences[2905071]

In [ ]:
news_sentences[3079526]

In [ ]:
news_sentences[4015059]

Таких предложений, судя по подсчетам символов из них немного, а потому просто проигнорируем их.

Посмотрим, какие символы мы уже имеем в пунктуации:

In [ ]:
punctuation = string.punctuation
punctuation

Этот список надо дополнить символами `«`, `»`, `—`, `…`.

In [ ]:
punctuation = ''.join(list(punctuation) + ['«', '»', '—', '…'])
punctuation

Будем удалять те токены, которые состоят лишь из знаков пунктуации. Запишем результаты на диск.

In [ ]:
for sentence in tqdm(news_sentences):
    tokenized_sentence = nltk.tokenize.word_tokenize(
        sentence.replace('\t', ' ').replace('\n', ' '), language='russian'
    )
    cleaned_tokenized_sentence = [
        x for x in tokenized_sentence 
        if not re.fullmatch('[' + punctuation + ']+', x)
    ]
    
    with open(os.path.join(RESULT_PATH, 'news_left_right.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence) + '\n')
        
    with open(os.path.join(RESULT_PATH, 'news_right_left.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence[::-1]) + '\n')

In [ ]:
del news_sentences, news_texts
gc.collect()

### Соцсети

Теперь обработаем тексты из соцсетей. Насчет включения этого раздела я до сих пор сомневаюсь. Тут весьма специфичный вокабуляр и достаточно много опечаток самих по себе.

Особенность обработки в том, что во всех источниках кроме Live Journal разные записи обозначаются при помощи DataBaseItem. Надо будем уметь детектировать разные записи и обрабатывать их отдельно.

In [ ]:
SOCIAL_PATH = os.path.join(TAIGA_PATH, 'social', 'texts')
social_texts = []

Начнем с Live Journal. При визуальном осмотре удалось заметить несколько особенностей:
1. Некоторые предложения очень короткие. Возможно, их стоит выбросить.
2. Часто повторяется строчка +100 -- выбросим ее.
3. Достаточно часто попадается построка `&quot` -- выбросим ее.

In [ ]:
lj_path = os.path.join(SOCIAL_PATH, 'LiveJournalPostsandcommentsGICR.txt')
social_texts = []
with open(lj_path, 'r') as inf:
    social_texts += inf.readlines()
    
social_texts = [x.replace('&quot', '') for x in social_texts if x != '+100\n']

Тексты из всех остальных источников обрабатываются одинаково, надо лишь пропустить строки, обозначающие DataBaseItem.

In [ ]:
for source in sorted(os.listdir(SOCIAL_PATH)):
    if source == 'LiveJournalPostsandcommentsGICR.txt':
        continue
    with open(os.path.join(SOCIAL_PATH, source), 'r') as inf:
        social_texts += inf.readlines()

social_texts = [x for x in social_texts if 'DataBaseItem' not in x]

Попробуем обработать обращения по имени из ВК. Для этого требуется при помощи регулярных выражений уловить конструкцию `[id|name]` и удалить там `id`. Со всем остальным справится токенизатор.

In [ ]:
social_texts = [re.sub('id[0-9]+', '', x) for x in social_texts]
social_texts = [re.sub('\*id\w+', '', x) for x in social_texts]

В некоторых строчках попадаются больше одного предложения. Надо их токенизировать.

In [ ]:
social_sentences = []
for text in tqdm(social_texts):
    social_sentences += [
        x.lower() for x in nltk.tokenize.sent_tokenize(
            text, language='russian'
        )
    ]

Осталось токенизировать предложения и выполнить запись на диск. Также отберем только те предложения, которые состоят по крайней мере из 5 слов.

In [ ]:
len_filter_border = 5
for sentence in tqdm(social_sentences):
    tokenized_sentence = nltk.tokenize.word_tokenize(
        sentence.replace('\t', ' ').replace('\n', ' '), language='russian'
    )
    cleaned_tokenized_sentence = [
        x for x in tokenized_sentence 
        if not re.fullmatch('[' + punctuation + ']+', x)
    ]
    if len(cleaned_tokenized_sentence) < len_filter_border:
        continue
    with open(os.path.join(RESULT_PATH, 'social_left_right.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence) + '\n')
        
    with open(os.path.join(RESULT_PATH, 'social_right_left.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence[::-1]) + '\n')

In [ ]:
del social_sentences
gc.collect()

### Субтитры

Обработаем тексты из субтитров.

Особенность обработки в том, что в данных помимо текста есть таймкоды. Также одно и то же предложение в общем случае разбито на несколько таймкодов.

Начнем с того, что загрузим таблицу с метаданными, чтобы доставать файлы с русскими субтитрами.

In [ ]:
SUBTITLES_PATH = os.path.join(TAIGA_PATH, 'subtitles')
subtitles_texts = []

In [ ]:
subtitles_df = pd.read_csv(os.path.join(SUBTITLES_PATH, 'metatable.csv'), sep='\t')
subtitles_df.head()

In [ ]:
subtitles_df = subtitles_df[subtitles_df['languages'] == 'ru']
subtitles_df.head()

Надо отдельно обработать случай сериала `Marvels Agents of S.H.I.E.L.D`. Дело в том, что данные между названиями второго и первого сезонов неконсистентны и это не полностью отражено в таблице (есть вариант написания `Marvel s Agents of S.H.I.E.L.D`).

In [ ]:
subtitles_df[subtitles_df['filepath'].str.startswith('Marvel')].head()

In [ ]:
filenames = [x.replace('Marvel s', 'Marvels') for x in subtitles_df['filepath'].tolist()]
folders = [x.split(' - ')[0].strip(' .') for x in filenames]

Отметим несколько особенностей для извелечения именно текста:
1. Вытащить именно текст вместо временных меток можно при помощи `split` по табам. 
2. Лучше сконкатенировать все строчки, так как иногда текст переносится, как уже было отмечено выше.
3. На последней строчке, насколько я мог наблюдать, расположены опции по отрисовке субтитров, а потому ее можно проигнорировать.

In [ ]:
for folder, filename in tqdm(zip(folders, filenames), total=len(folders)):
    with open(os.path.join(SUBTITLES_PATH, 'texts', folder, filename), 'r') as inf:
        subtitles_texts.append(
            ' '.join([x.split('\t')[-1].strip() for x in inf.readlines()][:-1])
        )

Токенизируем тексты.

In [ ]:
subtitles_sentences = []
for text in tqdm(subtitles_texts):
    subtitles_sentences += [
        x.lower() for x in nltk.tokenize.sent_tokenize(
            text, language='russian'
        )
    ]

Осталось токенизировать предложения и выполнить запись на диск. Также отберем только те предложения, которые состоят по крайней мере из 5 слов.

In [ ]:
len_filter_border = 5
lengths = []
for sentence in tqdm(subtitles_sentences):
    tokenized_sentence = nltk.tokenize.word_tokenize(
        sentence.replace('\t', ' ').replace('\n', ' '), language='russian'
    )
    cleaned_tokenized_sentence = [
        x for x in tokenized_sentence 
        if not re.fullmatch('[' + punctuation + ']+', x)
    ]
    if len(cleaned_tokenized_sentence) < len_filter_border:
        continue
    with open(os.path.join(RESULT_PATH, 'subtitles_left_right.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence) + '\n')
        
    with open(os.path.join(RESULT_PATH, 'subtitles_right_left.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence[::-1]) + '\n')

In [ ]:
del subtitles_sentences, subtitles_df
gc.collect()

### Сборка обучающего датасета

Теперь сконкатенируем полученные файлы для обучения языковых моделей.

In [7]:
!cat ../data/processed/kenlm/news_left_right.txt ../data/processed/kenlm/social_left_right.txt ../data/processed/kenlm/subtitles_left_right.txt > ../data/processed/kenlm/left_right.txt

In [8]:
!cat ../data/processed/kenlm/news_right_left.txt ../data/processed/kenlm/social_right_left.txt ../data/processed/kenlm/subtitles_right_left.txt > ../data/processed/kenlm/right_left.txt

Посмотрим на объем полученных датасетов.

In [9]:
!du ../data/processed/kenlm/left_right.txt -h

1.8G	../data/processed/kenlm/left_right.txt


In [10]:
!du ../data/processed/kenlm/right_left.txt -h

1.8G	../data/processed/kenlm/right_left.txt


## Обучение

Теперь выполним обучение. Для этого вспользуемя [документацией](https://kheafield.com/code/kenlm/estimation/) и [инструкцией](https://github.com/kmario23/KenLM-training).

На этом этапе подразумевается, что библиотека уже склонирована в src/kenlm и собрана.

In [13]:
! ../src/kenlm/build/bin/lmplz -o 3 --discount_fallback < ../data/processed/kenlm/left_right.txt > ../models/kenlm/left_right.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /home/mrgeekman/Documents/MIPT/НИР/Repo/data/processed/kenlm/left_right.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 155911769 types 2420218
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:29042616 2:3398434816 3:6372065280
Substituting fallback discounts for order 2: D1=0.5 D2=1 D3+=1.5
Statistics:
1 2420218 D1=0.76322 D2=0.928264 D3+=1.11566
2 32860060 D1=0.800531 D2=1.08166 D3+=1.2996
3 79416358 D1=0.5 D2=1 D3+=1.5
Memory estimate for binary LM:
type      MB
probing 2175 assuming -p 1.5
probing 2372 assuming -r models -p 1.5
trie     995 without quantization
trie     594 assuming -q 8 -b 8 quantization 
trie     922 assuming -a 22 array pointer compression
trie     520 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5

In [14]:
! ../src/kenlm/build/bin/lmplz -o 3 --discount_fallback < ../data/processed/kenlm/right_left.txt > ../models/kenlm/right_left.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /home/mrgeekman/Documents/MIPT/НИР/Repo/data/processed/kenlm/right_left.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 155911769 types 2420218
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:29042616 2:3398434816 3:6372065280
Substituting fallback discounts for order 2: D1=0.5 D2=1 D3+=1.5
Statistics:
1 2420218 D1=0.760231 D2=0.915235 D3+=1.14722
2 32860060 D1=0.798355 D2=1.06966 D3+=1.27319
3 79416358 D1=0.5 D2=1 D3+=1.5
Memory estimate for binary LM:
type      MB
probing 2175 assuming -p 1.5
probing 2372 assuming -r models -p 1.5
trie     995 without quantization
trie     594 assuming -q 8 -b 8 quantization 
trie     922 assuming -a 22 array pointer compression
trie     520 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3

К сожалению, пришлось добавить опцию `--discount_fallback`, потому что при использовании 3-грамм падает ошибка. По всей видимости, ему не хватает полученных данных.

### Бинаризация

Бинаризуем модель, чтобы ей можно было быстрее пользоваться.

In [15]:
!../src/kenlm/build/bin/build_binary ../models/kenlm/left_right.arpa ../models/kenlm/left_right.arpa.binary

Reading ../models/kenlm/left_right.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [16]:
!../src/kenlm/build/bin/build_binary ../models/kenlm/right_left.arpa ../models/kenlm/right_left.arpa.binary

Reading ../models/kenlm/right_left.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


Удалим теперь небинаризованные модели.

In [17]:
!rm ../models/kenlm/left_right.arpa
!rm ../models/kenlm/right_left.arpa

## Тест

А теперь загрузим модель и попробуем применить ее к какому-либо предложению.

In [18]:
import kenlm

model_left_right = kenlm.LanguageModel(os.path.join(MODEL_PATH, 'kenlm', 'left_right.arpa.binary'))
model_right_left = kenlm.LanguageModel(os.path.join(MODEL_PATH, 'kenlm', 'right_left.arpa.binary'))

In [19]:
example = 'журналисты всегда все нагло беспардонно переврут'
example_reversed = ' '.join(example.split(' ')[::-1])

In [20]:
model_left_right.score(example)

-29.755603790283203

In [21]:
model_right_left.score(example_reversed)

-29.687007904052734